L’application utilisée par le banquier nous envoie les informations suivantes:

{
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

* Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
* Committez, et envoyez moi le lien pour que je retrouve facilement

In [5]:
from pandas import read_csv
import pandas as pd

In [6]:
df_exo = pd.read_csv('exo_tets.csv', index_col = 0)

In [8]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
NaN,NaN,0.017036,42,1,0,NaN,6,0,1,0,1


In [10]:
for row in [df_exo] :
    row['MonthlyIncome'].fillna(0,inplace=True)

In [11]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1


In [14]:
for row in [df_exo] :
    row['NumberOfDependents'].fillna(0,inplace=True)

In [15]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1


In [17]:
for row in [df_exo] :
    row['NumberOfTotalTime'] = row['NumberOfTime30-59DaysPastDueNotWorse'] + row ['NumberOfTimes90DaysLate'] + row['NumberOfTime60-89DaysPastDueNotWorse']

In [18]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTotalTime
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1,1


In [19]:
for row in [df_exo] :
    row['MonthlyDebt'] = row ['DebtRatio'] * row ['MonthlyIncome']

In [21]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTotalTime,MonthlyDebt
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1,1,0.0


In [22]:
for row in [df_exo]:
    row['MonthlyBalance'] = row ['MonthlyIncome'] - row ['MonthlyDebt']

In [23]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTotalTime,MonthlyDebt,MonthlyBalance
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1,1,0.0,0.0


In [24]:
for row in [df_exo]:
    row['RatioIncomeDependents'] = row['MonthlyIncome'] / row ['NumberOfDependents']

In [25]:
df_exo

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTotalTime,MonthlyDebt,MonthlyBalance,RatioIncomeDependents
NaN,NaN,0.017036,42,1,0,0.0,6,0,1,0,1,1,0.0,0.0,0.0


In [32]:
export_csv = df_exo.to_csv (r'exo_testset.csv', header=True)

In [33]:
import bigml.api
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [34]:
source = api.create_source('fulltrain_dataframe.csv')

In [35]:
api.ok(source)

True

In [37]:
dataset_train = api.create_dataset(source)

In [38]:
api.ok(dataset_train)

True

In [39]:
test_source = api.create_source('exo_testset.csv')

In [40]:
api.ok(test_source)

True

In [42]:
dataset_test = api.create_dataset(test_source)

In [43]:
api.ok(dataset_test)

True

In [44]:
model = api.create_ensemble(dataset_train, {"objective_field":"SeriousDlqin2yrs"})

In [45]:
api.ok(model)

True

In [46]:
batch_prediction = api.create_batch_prediction(model, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True})

In [47]:
api.ok(batch_prediction)

True

In [48]:
api.download_batch_prediction(batch_prediction,
    filename='exo_prediction.csv')

'exo_prediction.csv'

In [55]:
df_exo_final = pd.read_csv('exo_prediction.csv', index_col = 0)

In [54]:
df_exo_

NameError: name 'df_exo_prediction' is not defined